In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import simbench as sb
import warnings

# Get notebook directory and parent (project root)
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent

# Add project directory to Python path
sys.path.append(str(project_dir))

# Import prepare_results from results_prep
from src.results_prep import prepare_results

In [ ]:
# Set the maximum number of rows to display in total
pd.set_option('display.max_rows', 300)

# Set the minimum number of rows to display before truncating
pd.set_option('display.min_rows', 200)

# set the max columns to none
pd.set_option('display.max_columns', None)

In [ ]:
# Define necessary OPEX parameters
maintenance_percentage_trafo = 0.02 # Source: Verteilnetzstudie Hessen
maintenance_percentage_line = 0.01  # Source: Verteilnetzstudie Hessen

equity_rate = 0.0769  # Source: Eigenkapitalzinssatz für Neuinvestitionen inkl. Gewerbesteuer, BNetzA
equity_percentage = 0.4  # Source: Pauschale Eigenkapitalquote,BNetzA
debt_rate = 0.0171  # Source: Kalkulatorischer Fremdkapitalzinssatz 4. Periode, BNetzA

current_HP_percentage = 5.7 # Source: bdew Report "Wie heizt Deutschland"

HP_tariff_discount = 0.6 # $14a EnWG

sales_tax = 0.19

payback_period = 35  # Source: BNetzA, in years
discount_rate = (equity_percentage * equity_rate) + ((1 - equity_percentage) * debt_rate) # WACC calculation

In [ ]:
# Set parameters
params = {
    'notebook_dir': project_dir,
    'payback_period': payback_period,
    'discount_rate': discount_rate,
    'equity_rate': equity_rate,
    'equity_percentage': equity_percentage,
    'debt_rate': debt_rate,
    'maintenance_percentage_trafo': maintenance_percentage_trafo,
    'maintenance_percentage_line': maintenance_percentage_line,
    'hp_tariff_discount': HP_tariff_discount,
    'sales_tax': sales_tax,
}

# Prepare results DataFrame
results_df = prepare_results(**params)

In [ ]:
# Filter data for HV-mixed and HV-urban grids
hv_mixed_data = results_df[results_df['grid_code'] == 'HV-mixed'].copy()
hv_urban_data = results_df[results_df['grid_code'] == 'HV-urban'].copy()

# Calculate reinforcement cost per household
hv_mixed_data['cost_per_household'] = hv_mixed_data['total_reinforcement_cost'] / hv_mixed_data['total_households']
hv_urban_data['cost_per_household'] = hv_urban_data['total_reinforcement_cost'] / hv_urban_data['total_households']

# Calculate mean and standard deviation at each HP adoption level
hv_mixed_summary = hv_mixed_data.groupby('HP_percentage')['cost_per_household'].agg(['mean', 'std']).reset_index()
hv_urban_summary = hv_urban_data.groupby('HP_percentage')['cost_per_household'].agg(['mean', 'std']).reset_index()

# 0: Check statistical significance

In [ ]:
# Set confidence level and Z-score for alpha = 0.05
confidence_level = 0.95
z_score = stats.norm.ppf((1 + confidence_level) / 2)

# Define acceptable margin of error as a percentage of the mean (e.g., 5%)
margin_error_percentage = 0.05

# Calculate confidence interval and minimum sample size
def significance_check_and_sample_size(summary_df, data_df):
    # Determine the number of unique runs based on the unique values in the 'random_seed' column
    n = data_df['random_seed'].nunique()

    significance_results = []

    for _, row in summary_df.iterrows():
        hp_level = row['HP_percentage']
        mean_cost = row['mean']
        std_dev = row['std']

        # Calculate current margin of error
        margin_of_error = z_score * (std_dev / np.sqrt(n))

        # Check if the current runs are significant
        is_significant = margin_of_error <= (margin_error_percentage * mean_cost)

        # Calculate required sample size for desired margin of error
        desired_margin_of_error = margin_error_percentage * mean_cost
        required_sample_size = (z_score * std_dev / desired_margin_of_error) ** 2

        # Store results
        significance_results.append({
            'HP_percentage': hp_level,
            'mean_cost': mean_cost,
            'std_dev': std_dev,
            'current_margin_of_error': margin_of_error,
            'is_significant': is_significant,
            'required_sample_size': np.ceil(required_sample_size)  # Round up to the nearest whole number
        })

    # Convert to DataFrame for easy viewing
    return pd.DataFrame(significance_results)

# Run significance check and sample size calculation for HV-mixed and HV-urban summaries
hv_mixed_significance = significance_check_and_sample_size(hv_mixed_summary, hv_mixed_data)
hv_urban_significance = significance_check_and_sample_size(hv_urban_summary, hv_urban_data)

In [ ]:
# Display the results
print("Distribution Grid - rural: Significance and Required Sample Sizes:")
hv_mixed_significance[['HP_percentage', 'mean_cost', 'current_margin_of_error', 'is_significant', 'required_sample_size']]

In [ ]:
print("\nDistribution Grid - urban: Significance and Required Sample Sizes:")
hv_urban_significance[['HP_percentage', 'mean_cost', 'current_margin_of_error', 'is_significant', 'required_sample_size']]

# Model validation: Peak load increase

In [ ]:
hv_data = results_df[results_df['grid_level'] == 'HV']

# Get volume-adjusted mean values for total grid load
total_load = hv_data.groupby('HP_percentage').apply(
    lambda x: (x['HP_percentage_load_HH_only'] * x['total_households']).sum() / x['total_households'].sum()
).reset_index()
total_load.columns = ['HP_percentage', 'Total grid load (in MW)']

# Calculate average load per household in kW
avg_load = total_load.copy()
avg_load['Avg. load by household (in kW)'] = avg_load['Total grid load (in MW)'] * 1000 / hv_data['total_households'].mean()
avg_load = avg_load.drop('Total grid load (in MW)', axis=1)

# Merge the two metrics
final_table = pd.merge(total_load, avg_load, on='HP_percentage')

# Filter for HP percentages from 0 to 100 in steps of 5
final_table = final_table[final_table['HP_percentage'].isin(range(0, 101, 5))]

# Set HP_percentage as column names and transpose
final_table = final_table.set_index('HP_percentage').transpose()

# Round the values
final_table = final_table.round(2)

final_table

# 1. Grid reinforcement cost of rural and urban

In [ ]:
# Plotting
plt.figure(figsize=(12, 8))

# Calculate global min and max y-axis limits across both datasets, with a 5% margin
global_min_y = min(hv_mixed_summary['mean'].min(), hv_urban_summary['mean'].min())
global_max_y = max(hv_mixed_summary['mean'].max(), hv_urban_summary['mean'].max())
y_min = global_min_y
y_max = global_max_y * 1.05  # Adding 5% margin

# Plot stair-step for Distribution Grid - rural (HV-mixed) with corridor
plt.step(hv_mixed_summary['HP_percentage'], hv_mixed_summary['mean'], where='mid', color='#003359', label='Distribution Grid - rural: Mean Cost per Household')
plt.fill_between(hv_mixed_summary['HP_percentage'],
                 hv_mixed_summary['mean'] - hv_mixed_summary['std'],
                 hv_mixed_summary['mean'] + hv_mixed_summary['std'],
                 color='#003359', alpha=0.2, label='Distribution Grid - rural: Simulation Results Std')

# Plot stair-step for Distribution Grid - urban (HV-urban) with corridor
plt.step(hv_urban_summary['HP_percentage'], hv_urban_summary['mean'], where='mid', color='#A2AD00', label='Distribution Grid - urban: Mean Cost per Household')
plt.fill_between(hv_urban_summary['HP_percentage'],
                 hv_urban_summary['mean'] - hv_urban_summary['std'],
                 hv_urban_summary['mean'] + hv_urban_summary['std'],
                 color='#A2AD00', alpha=0.2, label='Distribution Grid - urban: Simulation Results Std')

# Labels and title
plt.xlabel('HP Adoption Level (%)')
plt.ylabel('Reinforcement Cost per Household (€)')

# Set x-axis and y-axis limits
plt.xlim(0, 100)
plt.ylim(0, 1800)
    
plt.title('Reinforcement Cost per Household by Heat Pump Adoption Level')
plt.legend()
plt.grid(True)
plt.show()

# 1a: Distribution of reinforcement cost

In [ ]:
fig = plt.figure(figsize=(15, 10))

# Plot for transformers (upper subplot)
hv_data_rural = hv_data[hv_data['grid_code'] == 'HV-mixed']
hv_data_urban = hv_data[hv_data['grid_code'] == 'HV-urban']

# Calculate average trafo cost rural
rural_trafo_data = hv_data_rural.groupby('HP_percentage').agg({
    'total_households': 'mean',
    'LV_inc_trafo_reinforcement_cost': 'mean',
    'MV_inc_trafo_reinforcement_cost': 'mean',
}).reset_index()

rural_trafo_data['LV_inc_trafo_cost_per_household'] = (rural_trafo_data['LV_inc_trafo_reinforcement_cost'] / rural_trafo_data['total_households'])
rural_trafo_data['MV_inc_trafo_cost_per_household'] = (rural_trafo_data['MV_inc_trafo_reinforcement_cost'] / rural_trafo_data['total_households'])

# Calculate average trafo cost urban
urban_trafo_data = hv_data_urban.groupby('HP_percentage').agg({
    'total_households': 'mean',
    'LV_inc_trafo_reinforcement_cost': 'mean',
    'MV_inc_trafo_reinforcement_cost': 'mean',
}).reset_index()

urban_trafo_data['LV_inc_trafo_cost_per_household'] = (urban_trafo_data['LV_inc_trafo_reinforcement_cost'] / urban_trafo_data['total_households'])
urban_trafo_data['MV_inc_trafo_cost_per_household'] = (urban_trafo_data['MV_inc_trafo_reinforcement_cost'] / urban_trafo_data['total_households'])

# Calculate average line cost rural
rural_line_data = hv_data_rural.groupby('HP_percentage').agg({
    'total_households': 'mean',
    'LV_inc_line_reinforcement_cost': 'mean',
    'MV_inc_line_reinforcement_cost': 'mean',
    'HV_inc_line_reinforcement_cost': 'mean'
}).reset_index()

rural_line_data['LV_inc_line_cost_per_household'] = (rural_line_data['LV_inc_line_reinforcement_cost'] / rural_line_data['total_households'])
rural_line_data['MV_inc_line_cost_per_household'] = (rural_line_data['MV_inc_line_reinforcement_cost'] / rural_line_data['total_households'])
rural_line_data['HV_inc_line_cost_per_household'] = (rural_line_data['HV_inc_line_reinforcement_cost'] / rural_line_data['total_households'])

# Calculate average line cost urban
urban_line_data = hv_data_urban.groupby('HP_percentage').agg({
    'total_households': 'mean',
    'LV_inc_line_reinforcement_cost': 'mean',
    'MV_inc_line_reinforcement_cost': 'mean',
    'HV_inc_line_reinforcement_cost': 'mean'
}).reset_index()

urban_line_data['LV_inc_line_cost_per_household'] = (urban_line_data['LV_inc_line_reinforcement_cost'] / urban_line_data['total_households'])
urban_line_data['MV_inc_line_cost_per_household'] = (urban_line_data['MV_inc_line_reinforcement_cost'] / urban_line_data['total_households'])
urban_line_data['HV_inc_line_cost_per_household'] = (urban_line_data['HV_inc_line_reinforcement_cost'] / urban_line_data['total_households'])

# First row: Transformer plots
ax1 = plt.subplot(2, 2, 1)  # Rural transformers
ax2 = plt.subplot(2, 2, 2)  # Urban transformers

# Second row: Line plots
ax3 = plt.subplot(2, 3, 4)  # Rural lines LV
ax4 = plt.subplot(2, 3, 5)  # Rural lines MV
ax5 = plt.subplot(2, 3, 6)  # Rural lines HV

y_max = 150

# Plot transformer data for rural and urban grids
ax1.step(rural_trafo_data['HP_percentage'], rural_trafo_data['LV_inc_trafo_cost_per_household'], 
         where='mid', color='#003359', linestyle='-', label='Rural LV/MV')
ax1.step(urban_trafo_data['HP_percentage'], urban_trafo_data['LV_inc_trafo_cost_per_household'], 
         where='mid', color='#A2AD00', linestyle='-', label='Urban LV/MV')
ax1.set_title('LV/MV trafos')
ax1.set_xlabel('HP Adoption Level (%)')
ax1.set_ylabel('Marginal cost per household (€)')
ax1.grid(True)
ax1.legend()
ax1.set_ylim(0, y_max)  # Set same y-axis limit for transformer plots
ax1.set_xlim(0, 100)

ax2.step(rural_trafo_data['HP_percentage'], rural_trafo_data['MV_inc_trafo_cost_per_household'], 
         where='mid', color='#003359', linestyle='-', label='Rural MV/HV')
ax2.step(urban_trafo_data['HP_percentage'], urban_trafo_data['MV_inc_trafo_cost_per_household'], 
         where='mid', color='#A2AD00', linestyle='-', label='Urban MV/HV')
ax2.set_title('MV/HV trafos')
ax2.set_xlabel('HP Adoption Level (%)')
# ax2.set_ylabel('Marginal cost per household (€)')
ax2.grid(True)
ax2.legend()
ax2.set_ylim(0, y_max)  # Set same y-axis limit for transformer plots
ax2.set_xlim(0, 100)

# Plot line data
ax3.step(rural_line_data['HP_percentage'], rural_line_data['LV_inc_line_cost_per_household'], 
         where='mid', color='#003359', label='Rural LV')
ax3.step(urban_line_data['HP_percentage'], urban_line_data['LV_inc_line_cost_per_household'], 
         where='mid', color='#A2AD00', label='Urban LV')
ax3.set_title('LV lines')
ax3.set_xlabel('HP Adoption Level (%)')
ax3.set_ylabel('Marginal cost per household (€)')
ax3.grid(True)
ax3.legend()
ax3.set_ylim(0, y_max)  # Set same y-axis limit for line plots
ax3.set_xlim(0, 100)

ax4.step(rural_line_data['HP_percentage'], rural_line_data['MV_inc_line_cost_per_household'], 
         where='mid', color='#003359', label='Rural MV')
ax4.step(urban_line_data['HP_percentage'], urban_line_data['MV_inc_line_cost_per_household'], 
         where='mid', color='#A2AD00', label='Urban MV')
ax4.set_title('MV lines')
ax4.set_xlabel('HP Adoption Level (%)')
# ax4.set_ylabel('Marginal cost per household (€)')
ax4.grid(True)
ax4.legend()
ax4.set_ylim(0, y_max)  # Set same y-axis limit for line plots
ax4.set_xlim(0, 100)

ax5.step(rural_line_data['HP_percentage'], rural_line_data['HV_inc_line_cost_per_household'], 
         where='mid', color='#003359', label='Rural HV')
ax5.step(urban_line_data['HP_percentage'], urban_line_data['HV_inc_line_cost_per_household'], 
         where='mid', color='#A2AD00', label='Urban HV')
ax5.set_title('HV Lines')
ax5.set_xlabel('HP Adoption Level (%)')
# ax5.set_ylabel('Marginal cost per household (€)')
ax5.grid(True)
ax5.legend()
ax5.set_ylim(0, y_max)  # Set same y-axis limit for line plots
ax5.set_xlim(0, 100)

plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))

# First row: Transformer plots
ax1 = plt.subplot(2, 2, 1)  # Rural transformers
ax2 = plt.subplot(2, 2, 2)  # Urban transformers

# Second row: Line plots
ax3 = plt.subplot(2, 3, 4)  # Rural lines LV
ax4 = plt.subplot(2, 3, 5)  # Rural lines MV
ax5 = plt.subplot(2, 3, 6)  # Rural lines HV

y_max = 110

def plot_with_rolling_avg(ax, x, y, color, label, title, window=3):
    # Plot original data
    # ax.plot(x, y, color=color, alpha=0.3, linestyle='--', label=f'{label} (Raw)')
    
    # Calculate and plot rolling average
    rolling_avg = pd.Series(y).rolling(window=window, center=True).mean()
    ax.plot(x, rolling_avg, color=color, linewidth=2, label=f'{label}')
    ax.set_title(title)
    ax.set_xlabel('HP Adoption Level (%)')
    ax.set_ylabel('Marginal cost per household (€)')
    ax.grid(True)
    ax.legend()
    ax.set_ylim(0, y_max)
    ax.set_xlim(0, 100)

# Plot transformer data for rural and urban grids with rolling averages
plot_with_rolling_avg(ax1, rural_trafo_data['HP_percentage'], rural_trafo_data['LV_inc_trafo_cost_per_household'], 
                     '#003359', 'Rural', 'LV/MV trafos')
plot_with_rolling_avg(ax1, urban_trafo_data['HP_percentage'], urban_trafo_data['LV_inc_trafo_cost_per_household'], 
                     '#A2AD00', 'Urban', 'LV/MV trafos')

plot_with_rolling_avg(ax2, rural_trafo_data['HP_percentage'], rural_trafo_data['MV_inc_trafo_cost_per_household'], 
                     '#003359', 'Rural', 'MV/HV trafos')
plot_with_rolling_avg(ax2, urban_trafo_data['HP_percentage'], urban_trafo_data['MV_inc_trafo_cost_per_household'], 
                     '#A2AD00', 'Urban', 'MV/HV trafos')

# Plot line data with rolling averages
plot_with_rolling_avg(ax3, rural_line_data['HP_percentage'], rural_line_data['LV_inc_line_cost_per_household'], 
                     '#003359', 'Rural', 'LV lines')
plot_with_rolling_avg(ax3, urban_line_data['HP_percentage'], urban_line_data['LV_inc_line_cost_per_household'], 
                     '#A2AD00', 'Urban', 'LV lines')

plot_with_rolling_avg(ax4, rural_line_data['HP_percentage'], rural_line_data['MV_inc_line_cost_per_household'], 
                     '#003359', 'Rural', 'MV lines')
plot_with_rolling_avg(ax4, urban_line_data['HP_percentage'], urban_line_data['MV_inc_line_cost_per_household'], 
                     '#A2AD00', 'Urban', 'MV lines')

plot_with_rolling_avg(ax5, rural_line_data['HP_percentage'], rural_line_data['HV_inc_line_cost_per_household'], 
                     '#003359', 'Rural', 'HV lines')
plot_with_rolling_avg(ax5, urban_line_data['HP_percentage'], urban_line_data['HV_inc_line_cost_per_household'], 
                     '#A2AD00', 'Urban', 'HV lines')

In [ ]:
rural_output = hv_mixed_data.copy()
rural_output['grid_type'] = 'Distribution Grid - rural'

# Group and aggregate correctly
rural_output = rural_output.groupby('HP_percentage').agg({
    'total_households': 'mean',
    'total_reinforcement_cost': 'mean',
    'LV_total_reinforcement_cost': 'mean',
    'LV_total_trafo_reinforcement_cost': 'mean', 
    'LV_total_line_reinforcement_cost': 'mean',
    'MV_total_reinforcement_cost': 'mean', 
    'MV_total_trafo_reinforcement_cost': 'mean',
    'MV_total_line_reinforcement_cost': 'mean',
    'HV_total_reinforcement_cost': 'mean',
    'HV_total_line_reinforcement_cost': 'mean'}).reset_index()

# Divide all columns (except 'HP_percentage' and 'total_households') by 'total_households'
columns_to_divide = [col for col in rural_output.columns if col not in ['HP_percentage', 'total_households']]
for col in columns_to_divide:
    rural_output[f"{col}_per_hh"] = rural_output[col] / rural_output['total_households']

# Drop original columns if desired (excluding HP_percentage and total_households)
rural_output = rural_output[['HP_percentage', 'total_households'] + [f"{col}_per_hh" for col in columns_to_divide]]

urban_output = hv_urban_data.copy()
urban_output['grid_type'] = 'Distribution Grid - rural'

# Group and aggregate correctly
urban_output = urban_output.groupby('HP_percentage').agg({
    'total_households': 'mean',
    'total_reinforcement_cost': 'mean',
    'LV_total_reinforcement_cost': 'mean',
    'LV_total_trafo_reinforcement_cost': 'mean', 
    'LV_total_line_reinforcement_cost': 'mean',
    'MV_total_reinforcement_cost': 'mean', 
    'MV_total_trafo_reinforcement_cost': 'mean',
    'MV_total_line_reinforcement_cost': 'mean',
    'HV_total_reinforcement_cost': 'mean',
    'HV_total_line_reinforcement_cost': 'mean'}).reset_index()

# Divide all columns (except 'HP_percentage' and 'total_households') by 'total_households'
columns_to_divide = [col for col in urban_output.columns if col not in ['HP_percentage', 'total_households']]
for col in columns_to_divide:
    urban_output[f"{col}_per_hh"] = urban_output[col] / urban_output['total_households']

# Drop original columns if desired (excluding HP_percentage and total_households)
urban_output = urban_output[['HP_percentage', 'total_households'] + [f"{col}_per_hh" for col in columns_to_divide]]

# Add grid_code to each DataFrame
rural_output['grid_code'] = 'Distribution Grid - rural'
urban_output['grid_code'] = 'Distribution Grid - urban'

# Combine the DataFrames
combined_output = pd.concat([rural_output, urban_output], ignore_index=True)

In [ ]:
# Split the data by grid code
rural_costs = combined_output[combined_output['grid_code'] == 'Distribution Grid - rural'][
    ['HP_percentage',
     'LV_total_trafo_reinforcement_cost_per_hh',
     'MV_total_trafo_reinforcement_cost_per_hh',
     'LV_total_line_reinforcement_cost_per_hh',
     'MV_total_line_reinforcement_cost_per_hh',
     'HV_total_line_reinforcement_cost_per_hh'
    ]]

urban_costs = combined_output[combined_output['grid_code'] == 'Distribution Grid - urban'][
    ['HP_percentage',
     'LV_total_trafo_reinforcement_cost_per_hh',
     'MV_total_trafo_reinforcement_cost_per_hh',
     'LV_total_line_reinforcement_cost_per_hh',
     'MV_total_line_reinforcement_cost_per_hh',
     'HV_total_line_reinforcement_cost_per_hh'
    ]]

# Add a blank column between specified columns for rural costs
cols = list(rural_costs.columns)
insert_pos = cols.index('LV_total_line_reinforcement_cost_per_hh')
rural_costs.insert(insert_pos, 'blank', 0.000001)

# Add a blank column between specified columns for urban costs
cols = list(urban_costs.columns)
insert_pos = cols.index('LV_total_line_reinforcement_cost_per_hh') 
urban_costs.insert(insert_pos, 'blank', 0.000001)

# Drop the rows with HP_percentage = 0 from both rural and urban costs
rural_costs = rural_costs[rural_costs['HP_percentage'] != 0]
urban_costs = urban_costs[urban_costs['HP_percentage'] != 0]

# Transpose the dataframes to have HP percentages as columns
rural_costs_t = rural_costs.transpose()
urban_costs_t = urban_costs.transpose()

# Use HP_percentage as column names
rural_costs_t.columns = rural_costs_t.loc['HP_percentage']
urban_costs_t.columns = urban_costs_t.loc['HP_percentage']

# Drop the HP_percentage row since it's now used as column names
rural_costs_t = rural_costs_t.drop('HP_percentage')
urban_costs_t = urban_costs_t.drop('HP_percentage')

In [ ]:
rural_costs_t

In [ ]:
urban_costs_t

# 1b: Urban vs. rural

In [ ]:
# Define the original grid codes
original_grid_codes = ["1-HV-mixed--0-no_sw", "1-HV-urban--0-no_sw"]

# Clean the grid codes by removing the first 2 characters and the last 9 characters
cleaned_grid_codes = [code[2:-9] for code in original_grid_codes]

# Initialize an empty list to store DataFrames
line_lengths_list = []

# Calculate the average number of households for each cleaned grid code in results_df
households_summary = results_df[results_df['grid_code'].isin(cleaned_grid_codes)].groupby('grid_code')['total_households'].mean().reset_index()
households_summary.rename(columns={'total_households': 'average_total_households'}, inplace=True)

for code, cleaned_code in zip(original_grid_codes, cleaned_grid_codes):
    # Get the network for each original grid code
    net = sb.get_simbench_net(code)
    
    # Group by 'type' and sum the 'length_km'
    line_lengths = net.line.groupby('type')['length_km'].sum().to_frame()
    
    # Calculate the total length for this grid code
    total_length = line_lengths['length_km'].sum()
    
    # Calculate the percentage of each line type's length
    line_lengths['percentage'] = (line_lengths['length_km'] / total_length) * 100
    
    # Add columns for the cleaned grid code and total line length
    line_lengths['grid_code'] = cleaned_code  # Use cleaned code here
    line_lengths['total_length'] = total_length
    
    # Append to the list
    line_lengths_list.append(line_lengths)

# Concatenate all results into a single DataFrame
combined_line_lengths = pd.concat(line_lengths_list).reset_index()

# Merge with households_summary to add the average total households for each grid code
combined_line_lengths = combined_line_lengths.merge(households_summary, on='grid_code', how='left')

combined_line_lengths["km_line_per_household"] = combined_line_lengths["total_length"] / combined_line_lengths["average_total_households"]

# Display the final DataFrame
combined_line_lengths [["grid_code", "type", "length_km", "percentage", "km_line_per_household"]]